# Grouping Sets

In [11]:
SELECT gac.ParentProductCategoryName, gac.ProductCategoryName, COUNT(p.ProductID) AS Products
FROM SalesLT.Product AS p
JOIN SalesLT.vGetAllCategories AS gac 
ON p.ProductCategoryID = gac.ProductCategoryID
GROUP BY gac.ParentProductCategoryName, gac.ProductCategoryName 
-- GROUP BY GROUPING SETS(gac.ParentProductCategoryName, gac.ProductCategoryName, ())   -- Grouping sets are used to define custom groupings
-- GROUP BY ROLLUP(gac.ParentProductCategoryName, gac.ProductCategoryName)  -- ROLLUP includes subtotals and a grand total for hierarchical groupings
-- GROUP BY CUBE(gac.ParentProductCategoryName, gac.ProductCategoryName)   -- CUBE includes all possible groupings
ORDER BY gac.ParentProductCategoryName, gac.ProductCategoryName

(37 rows affected)

Total execution time: 00:00:00.013

ParentProductCategoryName,ProductCategoryName,Products
Accessories,Bike Racks,1
Accessories,Bike Stands,1
Accessories,Bottles and Cages,3
Accessories,Cleaners,1
Accessories,Fenders,1
Accessories,Helmets,3
Accessories,Hydration Packs,1
Accessories,Lights,3
Accessories,Locks,1
Accessories,Panniers,1


# Pivoting Data

In [43]:
SELECT * FROM (
    SELECT p.ProductID, pc.Name AS Category, ISNULL(p.Color, 'Uncolored') AS Color
    FROM SalesLT.Product AS p
    JOIN SalesLT.ProductCategory AS pc 
    ON p.ProductCategoryID = pc.ProductCategoryID
) AS ProdCats
PIVOT (COUNT(ProductID) FOR Color IN ([Black], [Blue], [Grey], [Red], [Silver], [Silver/Black], [White], [Yellow], [Multi], [Uncolored])) AS PivotTable
ORDER BY Category

(37 rows affected)

Total execution time: 00:00:00.007

Category,Black,Blue,Grey,Red,Silver,Silver/Black,White,Yellow,Multi,Uncolored
Bib-Shorts,0,0,0,0,0,0,0,0,3,0
Bike Racks,0,0,0,0,0,0,0,0,0,1
Bike Stands,0,0,0,0,0,0,0,0,0,1
Bottles and Cages,0,0,0,0,0,0,0,0,0,3
Bottom Brackets,0,0,0,0,0,0,0,0,0,3
Brakes,0,0,0,0,2,0,0,0,0,0
Caps,0,0,0,0,0,0,0,0,1,0
Chains,0,0,0,0,1,0,0,0,0,0
Cleaners,0,0,0,0,0,0,0,0,0,1
Cranksets,3,0,0,0,0,0,0,0,0,0


In [47]:
-- Unpivoting looses some details from original table

-- Create temporary table to archive the pivoted table
CREATE TABLE #ProductColorPivot
(Category varchar(20), Black int, Blue int, Grey int, Red int, Silver int, [Silver/Black] int, White int, Yellow int, Multi int, Uncolored int)

INSERT INTO #ProductColorPivot
SELECT * FROM (
    SELECT p.ProductID, pc.Name AS Category, ISNULL(p.Color, 'Uncolored') AS Color
    FROM SalesLT.Product AS p
    JOIN SalesLT.ProductCategory AS pc 
    ON p.ProductCategoryID = pc.ProductCategoryID
) AS ProdCats
PIVOT (COUNT(ProductID) FOR Color IN ([Black], [Blue], [Grey], [Red], [Silver], [Silver/Black], [White], [Yellow], [Multi], [Uncolored])) AS PivotTable
ORDER BY Category

(37 rows affected)

Total execution time: 00:00:00.013

In [52]:
-- Unpivot using temporary table
SELECT Category, Color, ProductCount FROM (
    SELECT Category, Black, Blue, Grey, Red , Silver, [Silver/Black], White, Yellow, Multi, Uncolored
    FROM #ProductColorPivot
) AS ProdColorCounts
UNPIVOT(ProductCount FOR Color IN ([Black], [Blue], [Grey], [Red], [Silver], [Silver/Black], [White], [Yellow], [Multi], [Uncolored])) AS ProductCount

(370 rows affected)

Total execution time: 00:00:00.028

Category,Color,ProductCount
Bib-Shorts,Black,0
Bib-Shorts,Blue,0
Bib-Shorts,Grey,0
Bib-Shorts,Red,0
Bib-Shorts,Silver,0
Bib-Shorts,Silver/Black,0
Bib-Shorts,White,0
Bib-Shorts,Yellow,0
Bib-Shorts,Multi,3
Bib-Shorts,Uncolored,0
